# Analysis of extrinsic results

TODO: Summary of extrinsic 3a and 3b?
- everything in extrinsic summaries - read results, redo tables

TODO: Analysis of spatial autocorrelation of:

- differences in density
- matched/unmatched features
- connectivity
- components per grid cell?

TODO: Analysis of overlap between errors and differences.

TODO: Analysis of correlation between high/low quality areas and pop density

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pickle
import json
import pandas as pd
import seaborn as sns
import plotly.express as px

## Differences per municipality

In [27]:
# Load results of municipal analysis

osm_muni = pd.read_csv("../results/osm_quality/muni_network_counts.csv",index_col=False)
ref_muni = pd.read_csv("../results/geodk_quality/muni_network_counts.csv",index_col=False)

osm_muni.set_index('navn',inplace=True)
ref_muni.set_index('navn',inplace=True)

osm_muni.fillna(0,inplace=True)
ref_muni.fillna(0,inplace=True)

# TODO: Compute components per km!
# TODO: convert infra length to km?

osm_muni['datasource'] = 'osm'
ref_muni['datasource'] = 'ref'

diffs = []

for c in osm_muni.columns:
    diff = osm_muni[c] - ref_muni[c]
    diffs.append(diff)

diff_df = pd.DataFrame(diffs).transpose()
diff_df

In [ ]:
diff_df.describe()

In [3]:
osm_muni.compare(ref_muni)

infrastructure_length                node_count       dangling_node_count  \
                    self          other       self other                self   
0          335114.384014  193541.697984       1238  1390                 751   
1          670714.668671  218764.628637       3708  1478                1841   
2          937870.720797  240556.379891       5153  2097                2391   
3          110961.318630    9893.782706        509    86                 196   
4          157756.329035   54391.846843       1150   297                 512   
..                   ...            ...        ...   ...                 ...   
93         453020.522640  203558.475958       2330  1002                1264   
94         295317.880306   41079.393930        766   221                 384   
95         453235.159585  401855.089934       2277  1199                1147   
96         180389.972247   75289.385711        602   363                 324   
97          15981.724117    6246.945147         34    48                  16   

         infra_km          infra_dens         ... overshoots        \
   other     self    other       self  other  ...       self other   
0    126  335.114  193.542      0.356  0.206  ...        2.0   NaN   
1    306  670.715  218.765      0.589  0.192  ...        2.0   NaN   
2    690  937.871  240.556      1.990  0.511  ...        7.0   3.0   
3     62  110.961    9.894      4.734  0.422  ...        2.0   1.0   
4     69  157.756   54.392      2.340  0.807  ...        2.0   1.0   
..   ...      ...      ...        ...    ...  ...        ...   ...   
93   219  453.021  203.558      0.427  0.192  ...        NaN   NaN   
94    62  295.318   41.079      0.385  0.053  ...        NaN   NaN   
95   256  453.235  401.855      0.319  0.283  ...        5.0   1.0   
96   109  180.390   75.289      0.289  0.120  ...        NaN   NaN   
97     2   15.982    6.247      0.178  0.070  ...        NaN   NaN   

   undershoots       over_under       component_gaps        component_count  \
          self other       self other           self  other            self   
0          NaN   6.0        NaN   NaN            9.0   32.0           230.0   
1          8.0  12.0       10.0   0.0           41.0   66.0           444.0   
2         14.0  32.0       21.0  35.0           85.0  257.0           430.0   
3          1.0   NaN        3.0   0.0            8.0    1.0             NaN   
4          5.0   4.0        7.0   5.0           23.0   32.0            86.0   
..         ...   ...        ...   ...            ...    ...             ...   
93         1.0  21.0        2.0  22.0           25.0   90.0           285.0   
94         NaN   NaN        NaN   NaN            9.0   24.0            95.0   
95         NaN   4.0        0.0   5.0           18.0  127.0           298.0   
96         NaN   6.0        NaN   NaN           10.0   73.0            92.0   
97         NaN   NaN        NaN   NaN            1.0    0.0             8.0   

           
    other  
0    51.0  
1   102.0  
2   271.0  
3     NaN  
4    30.0  
..    ...  
93   86.0  
94   26.0  
95  112.0  
96   54.0  
97    3.0  

[98 rows x 22 columns]

In [32]:
combined = pd.concat([osm_muni.reset_index(), ref_muni.reset_index()])

px.violin(combined, x='datasource', y='infrastructure_length')

,navn,infrastructure_length,node_count,dangling_node_count,infra_km,area_sqkm,infra_dens,pop,infra_pop,overshoots,undershoots,over_under,component_gaps,component_count,datasource
0,Aabenraa,335114.384014,1238,751,335.114,940.61,0.356,59002,5.679706,2.0,0.0,0.0,9,230,osm
1,Aalborg,670714.668671,3708,1841,670.715,1138.00,0.589,222571,3.013488,2.0,8.0,10.0,41,444,osm
2,Aarhus,937870.720797,5153,2391,937.871,471.20,1.990,361544,2.594072,7.0,14.0,21.0,85,430,osm
3,Albertslund,110961.318630,509,196,110.961,23.44,4.734,27530,4.030548,2.0,1.0,3.0,8,30,osm
4,Allerød,157756.329035,1150,512,157.756,67.43,2.340,26061,6.053336,2.0,5.0,7.0,23,86,osm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Vejle,203558.475958,1002,219,203.558,1061.61,0.192,120949,1.683007,1.0,21.0,22.0,90,86,ref
94,Vesthimmerlands,41079.393930,221,62,41.079,768.04,0.053,36431,1.127584,0.0,0.0,0.0,24,26,ref
95,Viborg,401855.089934,1199,256,401.855,1421.26,0.283,97731,4.111848,1.0,4.0,5.0,127,112,ref
96,Vordingborg,75289.385711,363,109,75.289,625.26,0.120,45441,1.656852,0.0,6.0,0.0,73,54,ref


In [40]:
px.bar(diff_df, y='infrastructure_length', x=diff_df.index)

In [ ]:
# TODO; Make a new dataframe - all the differences

# TODO: Make a new dataframe - pct differences

# TODO: compare ranks?

# TODO: Plot correlation between diff